# 第十一讲：贝叶斯统计及正则化

我们今天将要介绍另一种避免过拟合的方法——正则化，该方法不需要剔除特征值。

## 3. 贝叶斯统计与正则化

在一开始，我们先来讨论使用最大似然（ML: Maximum Likelihood）拟合参数。以往，我们都是通过下面这个式子选择参数：

$$
\theta_\mathrm{ML}=\mathrm{arg}\operatorname*{max}_\theta\prod_{i=1}^mp\left(y^{(i)}\mid x^{(i)};\theta\right)
$$

在后面的讨论中，我们始终将$\theta$看做未知参数。从**频率学派（frequentist）**的观点看，$\theta$是一个*未知*的但却是*常量*的值。对于频率派来说，$\theta$并不是随机变量，它是一个参数，只是恰巧不知道它的值而已，而我们的任务就是通过使用一些统计步骤（如最大似然法）来估计这个参数的值。

从贝叶斯学派的观点看问题，则会找到另一种估计参数的方法——他们将$\theta$看做是一个*随机变量*，其值未知。于是，我们可以假设一个关于$\theta$的**先验分布（prior distribution）**$p(\theta)$，用来表示我们“相信”参数取值的不确定性。对于给定的训练集$S=\left\{\left(x^{(i)},y^{(i)}\right)\right\}_{i=1}^m$，当我们需要预测一个新的$x$时，我们就可以计算参数的后验分布：

$$
\begin{align}
p(\theta\mid S)&=\frac{p(S\mid\theta)p(\theta)}{p(S)}\\
&=\frac{\left(\prod_{i=1}^mp\left(y^{(i)}\mid x^{(i)},\theta\right)\right)p(\theta)}{\int_{\theta}\left(\prod_{i=1}^mp\left(y^{(i)}\mid x^{(i)},\theta\right)p(\theta)\right)\mathrm{d}\theta}\tag{1}
\end{align}
$$

式中的$p\left(y^{(i)}\mid x^{(i)},\theta\right)$来自我们在学习问题中选择的模型。举个例子，如果我们使用贝叶斯逻辑回归，那么$p\left(y^{(i)}\mid x^{(i)},\theta\right)=h_\theta\left(x^{(i)}\right)^{y^{(i)}}\left(1-h_\theta\left(x^{(i)}\right)\right)^{\left(1-y^{(i)}\right)}$，这里的$h_\theta\left(x^{(i)}\right)=\frac{1}{1+\exp\left(-\theta^Tx^{(i)}\right)}$。（因为我们已经把$\theta$看做是一个随机变量了，所以就可以把它写入条件概率的条件中，即使用$p(y\mid x;\theta)$代替$p(y\mid x,\theta)$。）

当我们需要预测新的$x$时，就可以使用关于$\theta$的后验分布计算相应标记类型$y$上的后验概率了：

$$
p(y\mid x,S)=\int_\theta p(y\mid x,\theta)p(\theta\mid S)\mathrm{d}\theta\tag{2}
$$

上式中的$p(\theta\mid S)$来自$(1)$式，举个例子，如果要预测给定$x$的标记类型$y$，则有（如果$y$是取离散值，则使用求和代替求积分）：

$$
\mathrm{E}[y\mid x,S]=\int_yyp(y\mid x,S)\mathrm{d}y
$$

上面简要的例子可以看做是“完全贝叶斯式的”预测，也就是计算$p(\theta\mid S)$关于$\theta$的后验概率的平均值。不过，这个后验分布通常难以计算，因为在$(1)$中需要求出关于$\theta$（通常是维度很高）的积分，一般难以求得这些积分的闭合形式（即解析解，相对于数值解而言）。

因此，在实践中，我们选择近似这个关于$\theta$的后验分布。一个常用的近似就是使用一个点的估计来代替这个后验分布。介绍一下关于$\theta$的**MAP（maximum a posteriori）**估计（也就是对$\theta$后验概率的最大化估计，因为后验概率正比于$(1)$式的分子，所以我们最大化这个分子）：

$$
\begin{align}
\theta_\mathrm{MAP}&=\mathrm{arg}\operatorname*{max}_\theta p(\theta\mid S)\\
&=\mathrm{arg}\operatorname*{max}_\theta\prod_{i=1}^mp\left(y^{(i)}\mid x^{(i)},\theta\right)p(\theta)\tag{3}
\end{align}
$$

与关于$\theta$的最大似然估计的式子相比，这个式子只是最后多了一项$p(\theta)$。（对于这个式子的直观理解，参数服从$\theta\sim\mathcal{N}\left(0,\tau^2I\right)$，这是一个期望为$0$的高斯分布，具有一定的协方差$\tau^2I$，对于这样一个分布，大部分的概率质量都集中在$0$附近。所以这个先验概率的意义就是，多数的参数都应该接近于$0$，回忆上一讲介绍的特征选择，如果我们剔除某个特征，也就等同于将其值置为$0$，这是一个与特征选择类似的过程，虽然它不会将大多数参数严格的置为$0$。我们用最大似然估计拟合参数时，如果是使用高次多项式拟合，那么得到的曲线将会有较大波动，而使用贝叶斯正则化方法时，得到的曲线将随着$\tau$的减小而越来越平滑，因为协方差减小时越来越多的特征值将向$0$靠近。）

（极大似然估计算法，比如线性回归，尝试最小化$\displaystyle\operatorname*{min}_\theta\sum_i\left\lVert y^{(i)}-\theta^Tx^{(i)}\right\rVert^2$。而现在加入先验概率后，相当于最小化$\displaystyle\operatorname*{min}_\theta\sum_i\left\lVert y^{(i)}-\theta^Tx^{(i)}\right\rVert^2+\lambda\lVert\theta\rVert^2$，也就是在最后加了一项，当参数过大时会惩罚目标函数。所以，这个算法与极大似然估计相似，但倾向于选择较小的参数，得到的拟合更加平滑。）

在实际应用中，对于先验概率$p(\theta)$来说，通常假设$\theta\sim\mathcal{N}\left(0,\tau^2I\right)$。与最大似然相比，通过使用先验拟合得到的参数$\theta_\mathrm{MAP}$具有较小的范数（即$\left\lVert\theta_\mathrm{MAP}\right\rVert^2\leq\left\lVert\theta_\mathrm{ML}\right\rVert^2$见[问题集3](http://cs229.stanford.edu/materials/ps3.pdf)）。在实际应用中，与ML相比，贝叶斯MAP估计更不容易受到过拟合影响。比如，在文本分类问题上即使当$n\gg m$时，贝叶斯逻辑回归也表现出良好的性能。

## 4. 感知及大间隔分类器

在关于学习理论的讲义的最后，我们来介绍一种不同的机器学习模型。到目前位置，我们介绍的方法都属于**批量学习（batch learning）**算法，也就是先使用训练集训练模型，再使用另一个集合中的测试数据来评估得到的的假设函数$h$。那么接下来，我们将介绍**在线学习（online learning）**算法，该算法在学习时也需要持续的做出预测。

对于给定的训练样本序列$\left(x^{(1)},y^{(1)}\right),\left(x^{(2)},y^{(2)}\right),\cdots,\left(x^{(m)},y^{(m)}\right)$，算法先看到$x^{(1)}$，然后要对其做出预测得到$\hat y^{(1)}$，在预测之后，算法才会知道$y^{(1)}$真实值（然后算法可能会根据这个真实值做学习调整）。接着，算法看到$x^{(2)}$，然后要再对这个输入做出预测得到$\hat y^{(2)}$，之后会得到$y^{(2)}$真实值，算法再根据这个真实值做出学习调整。这个过程会一直持续到$\left(x^{(m)},y^{(m)}\right)$。对于在线学习算法，我们关心整个学习过总在线误差（total online error）$\displaystyle\sum_{i=1}^m1\left\{\hat y^{(i)}\neq y^{(i)}\right\}$。因此，这个是一个即使在在学习的过程中也要做出预测的算法。

（其实我们以前的学习算法都可以使用在线学习，比如，在需要对$x^{(3)}$做出预测时，我们将前面的两个样本作为训练集使用即可。而对于某些算法，比如梯度下降法，可以更好的应用在线学习算法。）

我们先给感知算法的在线误差设一个界限。为了简化接下来的推导，我们使用管理标记$y\in\left\{-1,1\right\}$。

回忆具有参数$\theta\in\mathbb{R}^{n+1}$的感知算法，它按照下面的式子做出预测：

$h_\theta(x)=g\left(\theta^Tx\right)\tag{1}$

其中$g(z)=\begin{cases}1&\textrm{if }z\geq0\\-1&\textrm{if }z\lt0\end{cases}$。

对于给定的训练样本$(x,y)$，感知学习算法的更新规则为：

* 如果$h_\theta(x)=y$，则参数不更新；
* 否则，按规则$\theta:=\theta+yx$更新。（这与我们最初在[第三讲](chapter03.ipynb)中介绍的更新规则略有不同，因为我们现在使用$y\in\left\{-1,1\right\}$作为标记，同时也放弃使用学习速率参数$\alpha$。因为此时的学习速率只会按一个常量同时缩放所有的参数，而对感知算法的学习行为没有任何影响。）

下面的理论将会给感知算法的在线学习误差一个界限，当感知算法按照在线学习算法运行时，每次错误的预测都会引起参数更新。应当注意的是，下面这个理论中关于“误差个数的界限”同“序列中样本的总数$m$”或是“输入特征的维度$n$”并无明确的依赖关系。

**定理（Block, 1962, and Novikoff, 1962）；**给定样本序列$\left(x^{(1)},y^{(1)}\right),\left(x^{(2)},y^{(2)}\right),\cdots,\left(x^{(m)},y^{(m)}\right)$，假设对于所有样本$i$，有$\left\lVert x^{(i)}\right\rVert\leq D$，进一步假设存在单位向量$u$（$\lVert u\rVert^2=1$），对于序列中的所有样本有$y^{(i)}\cdot\left(u^Tx^{(i)}\right)\geq\gamma$（即当$y^{(i)}=1$时$\left(u^Tx^{(i)}\right)\geq\gamma$，当$y^{(i)}=-1$时$\left(u^Tx^{(i)}\right)\leq\gamma$，也就是$u$将数据分隔开，间隔至少为$\gamma$）。那么，感知算法的误差总数将不会超过$\left(\frac{D}{\gamma}\right)^2$。

**证明：**感知算法只有在预测错误时调整参数，令$\theta^{(k)}$为算法在第$k$次预测错误时所使用的参数，则$\theta^{(1)}=\vec0$（因为参数以$\vec0$初始化），而如果第$k$次预测错误发生在样本上$\left(x^{(i)},y^{(i)}\right)$，则$g\left(\left(x^{(i)}\right)^T\theta^{(k)}\right)\neq y^{(i)}$，于是有：

$$
\left(x^{(i)}\right)^T\theta^{(k)}y^{(i)}\leq0\tag{2}
$$

从感知更新规则可以得到$\theta^{(k+1)}=\theta^{(k)}+y^{(i)}x^{(i)}$，于是有：

$$
\begin{align}
\left(\theta^{(k+1)}\right)^Tu&=\left(\theta^{(k)}\right)^Tu+y^{(i)}\left(x^{(i)}\right)^Tu\\
&\geq\left(\theta^{(k)}\right)^T+\gamma
\end{align}
$$

通过数学归纳法可知：

$$
\left(\theta^{(k+1)}\right)^Tu\geq k\gamma\tag{3}
$$

同时，也有：

$$
\begin{align}
\left\lVert\theta^{(k+1)}\right\rVert^2&=\left\lVert\theta^{(k)}+y^{(i)}x^{(i)}\right\rVert^2\\
&=\left\lVert\theta^{(k)}\right\rVert^2+\left\lVert x^{(i)}\right\rVert^2+2y^{(i)}\left(x^{(i)}\right)^T\theta^{(i)}\\
&\leq\left\lVert\theta^{(k)}\right\rVert^2+\left\lVert x^{(i)}\right\rVert^2\\
&\leq\left\lVert\theta^{(k)}\right\rVert^2+D^2\tag{4}
\end{align}
$$

上面的第三步用到了$(2)$式，对$(4)$式再使用归纳法可得：

$$
\left\lVert\theta^{(k+1)}\right\rVert^2\leq kD^2\tag{5}
$$

结合$(3)$式和$(4)$式：

$$
\begin{align}
\sqrt kD&\geq\left\lVert\theta^{(k+1)}\right\rVert\\
&\geq\left(\theta^{(k+1)}\right)^Tu\\
&\geq k\gamma
\end{align}
$$

推导第二个不等式使用了单位向量的性质：$z^Tu=\left\lVert z\right\rVert\cdot\left\lVert u\right\rVert\cos\phi\leq\left\lVert z\right\rVert\cdot\left\lVert u\right\rVert$，其中$\phi$是向量$z$与向量$u$的夹角。结果表明$k\leq\left(\frac{D}{\gamma}\right)^2$。因此，如果感知算法第$k$次预测错误，则$k\leq\left(\frac{D}{\gamma}\right)^2$。

（所以，在使用感知算法时，即使输入特征$x^{(i)}\in\mathbb{R}^\infty$，或向量维数非常高，只要在这个高维空间中，正负样本能被某个间隔分开，该算法会收敛到一个能够完美分隔样本的边界上。）

## 5. 机器学习应用的一些建议

本节主要是[演示文稿](http://cs229.stanford.edu/materials/ML-advice.pdf)，有以下几点需要注意：

* 介绍如何将机器学习算法应用在不同的应用问题上的一些建议；
* 本节的材料并不涉及很多数学运算，但仍旧是本课程最难理解的一部分内容；
* 本节的内容在今天可能仍有争议；
* 本节的一些方法并不有利于新的学习算法的研发，只是有利于现有算法的应用；
* 本节的三个重点：
    1. 学习算法的调试诊断；
    2. 误差分析及销蚀分析；
    3. 如何开始用机器学习解决问题
        * 过早统计优化（premature statistical optimization，与软件工程中的过早优化类似，指对机器学习系统中一个非常不重要的部分进行过度的优化）

### 5.1 调试学习算法

举个例子，比如在写垃圾邮件分类器时，我们小心的选择了$100$个单词作为特征值。（而不是使用全部$50000+$的英语单词）。然后使用贝叶斯逻辑回归算法$\displaystyle\operatorname*{max}_\theta\sum_{i=1}^m\log p\left(y^{(i)}\mid x^{(i)},\theta\right)-\lambda\lVert\theta\rVert^2$，再使用梯度下降法算出结果。测试模型发现，得到了不可接受的20%的错误率。

接下来怎么办？

通常的想法是想方设法的优化这个算法，包括：
* 获得更多的训练样本；
* 寻找更小的特征值集合；
* 寻找更大的特征值集合；
* 尝试改变特征值，比如“从邮件头部获取信息”V.S.“从邮件正文获取信息”；
* 让算法再多迭代几次（怀疑算法没有完全收敛）；
* 尝试换用牛顿法计算参数；
* 给$\lambda$赋不同的值；
* 尝试换用SVM算法。

这里仅列出了八项，而在实际的学习系统中，能够列出上百项可行的改进方法，其中的某些尝试可能会有效（比如获取更多的训练样本），但显然非常浪费时间，而且非常需要运气，就算解决了也可能仍旧不知道问题出在哪。

所以，我们应该先诊断问题所在，然后再想方法解决。那么，对于上面的例子：

假设我们怀疑问题出现在：
* 过拟合（高方差）；
* 特征值过少，无法有效分类（高偏差）

诊断：
* 高方差的特点：训练误差比测试误差小很多；
* 高偏差的特点：训练误差也很高；

高方差的典型学习曲线：

<img src="./resource/chapter11_image01.png" width="500" alt="" align=center />

* 当训练集$m$增大时测试误差也会继续降低（而训练误差会持续增加，因为样本越多，越难以完美拟合），建议增加训练样本；
* 训练误差与测试误差中的差距过大。

高偏差的典型学习曲线：

<img src="./resource/chapter11_image02.png" width="500" alt="" align=center />

* 即使是训练误差也是高的不可思议（还可以发现增加训练样本的效果已经很不明显了，而此时增加训练样本只会使训练效果越来越差，因为通常的训练误差是关于训练样本的单调增函数）；
* 而训练误差与测试误差间的差距很小。

（所以，可以通过判断训练误差与测试误差间的距离判断）

于是，对于这个使用梯度下降法实现的贝叶斯逻辑回归，回到前面列出的八项解决方案：
* 获得更多的训练样本；*（有助于修复高方差）*
* 寻找更小的特征值集合；*（有助于修复高方差）*
* 寻找更大的特征值集合；*（有助于修复高偏差）*
* 尝试改变特征值，比如“从邮件头部获取信息”V.S.“从邮件正文获取信息”；*（有助于修复高偏差）*
* 让算法再多迭代几次（怀疑算法没有完全收敛）；
* 尝试换用牛顿法计算参数；
* 给$\lambda$赋不同的值；
* 尝试换用SVM算法。

上面这个关于方差与偏差的诊断是一个常见的机器学习调试方法。而对于别的问题则需要我们构造自己的诊断方法，找出问题。

我们再举一个例子：
* 使用贝叶斯逻辑回归时，对垃圾邮件存在$2%$的误判，对非垃圾邮件也存在$2%$的误判（也就是对正常邮件的误判率过高）；
* 而使用带有线性核的SVM时，对垃圾邮件存在$10%$的误判，对非垃圾邮件存在$0.01%$的误判（在正常范围内）；
* 因为运算效率的问题，我们更希望使用逻辑回归。

这次，又该怎么办？

\left(\right)
\begin{align}\end{align}
\left\lVert\right\rVert